In [1]:
%matplotlib inline
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib as mpl
import IPython.display as ipd
import sklearn as sk
import sklearn.metrics
import sklearn.model_selection

In [8]:
data = pd.read_csv('datasets/bikesharing-day.csv', index_col=0)

data.dteday = pd.to_datetime(data.dteday, format='%Y-%m-%d').dt.normalize()
data.season = data.season.astype('category')
data.mnth = data.mnth.astype('category')
if ('hr' in data):
    data.hr = data.hr.astype('category')
data.holiday = data.holiday.astype(bool)
data.weekday = data.weekday.astype('category')
data.workingday = data.workingday.astype(bool)
data.weathersit = data.weathersit.astype('category')

In [21]:
train, test = sk.model_selection.train_test_split(
    data.loc[:, ['temp', 'hum', 'cnt']]
)   

In [54]:
def regression_model(Model, predict):
    variates = train.columns.drop(predict)
    
    model = Model()
    model.fit(train.loc[:, variates], train.loc[:,predict])
    y_predicted = model.predict(test.loc[:, variates])
    y_true = test.loc[:,'cnt']
    return pd.Series({
        'RMSE': np.sqrt(sk.metrics.mean_squared_error(y_true,y_predicted)),
        'MAE': sk.metrics.mean_absolute_error(y_true,y_predicted),
        'model': model
    }, name=Model.__name__)

In [55]:
import sklearn.linear_model
regression_model(sk.linear_model.LinearRegression, 'cnt')

RMSE                                                1443.4
MAE                                                 1182.4
model    LinearRegression(copy_X=True, fit_intercept=Tr...
Name: LinearRegression, dtype: object